In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Install necessary libraries
#!pip install transformers datasets evaluate

# Import libraries
import os
import numpy as np
from transformers import ViTFeatureExtractor, TFAutoModelForImageClassification
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from datasets import load_dataset
from tensorflow.keras.optimizers import Adam

/Users/pauljohnmaddala/anaconda3/envs/tfenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def custom_data_generator(directory, batch_size, feature_extractor):
    datagen = ImageDataGenerator(rescale=1./255)
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            images = os.listdir(folder_path)[:50]  # Take first 50 images
            while True:
                for i in range(0, len(images), batch_size):
                    batch_images = images[i:i + batch_size]
                    batch_data = []
                    batch_labels = []
                    for img in batch_images:
                        img_path = os.path.join(folder_path, img)
                        image = load_img(img_path, target_size=(224, 224))
                        image = img_to_array(image)
                        image = datagen.standardize(image)
                        batch_data.append(feature_extractor(images=image, return_tensors="tf").pixel_values[0])

                        # Assuming label is the folder name
                        label_index = ord(folder.lower()) - ord('a')  # Convert folder name to a numerical label
                        batch_labels.append(label_index)

                    yield np.array(batch_data), np.array(batch_labels)


In [3]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = TFAutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=26)


/Users/pauljohnmaddala/anaconda3/envs/tfenv/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/kernel:0', 'vit/pooler/dense/bias:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImage

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
train_dir = 'DATASET/train'  # Replace with your training directory path
batch_size = 32
train_generator = custom_data_generator(train_dir, batch_size, feature_extractor)

steps_per_epoch = (10 * 29) // batch_size  # 26 classes, 50 images each
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=3)


Epoch 1/3


2023-11-30 16:51:20.121649: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


9/9 [==============================] - 109s 11s/step - loss: 3.5064 - accuracy: 0.6466
Epoch 2/3
9/9 [==============================] - 92s 11s/step - loss: 2.5034e-06 - accuracy: 1.0000
Epoch 3/3
9/9 [==============================] - 99s 11s/step - loss: 2.5034e-06 - accuracy: 1.0000


In [6]:
def custom_data_generator_for_test(directory, batch_size, feature_extractor):
    datagen = ImageDataGenerator(rescale=1./255)
    images = os.listdir(directory)[:50]  # Adjust the number of images as needed
    while True:
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i + batch_size]
            batch_data = []
            for img in batch_images:
                img_path = os.path.join(directory, img)
                image = load_img(img_path, target_size=(224, 224))
                image = img_to_array(image)
                image = datagen.standardize(image)
                batch_data.append(feature_extractor(images=image, return_tensors="tf").pixel_values[0])

            # Create dummy labels (e.g., zeros) since the test set might be unlabeled
            dummy_labels = np.zeros((len(batch_data),))  # Adjust the shape according to your model's requirement

            yield np.array(batch_data), dummy_labels


In [ ]:
import os
import math

test_dir = 'DATASET/test'  # Replace with your test directory path
test_generator = custom_data_generator_for_test(test_dir, batch_size, feature_extractor)

# Calculate the number of steps for the test data
# Ensure test_steps is at least 1 and round up to cover all data
num_test_images = len(os.listdir(test_dir))
test_steps = max(1, math.ceil(num_test_images / batch_size))

# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=test_steps)

print(f"Test Accuracy: {accuracy * 100:.2f}%")